<a href="https://colab.research.google.com/github/GoshaMareev/BDSA/blob/main/Advanced_PySpark_for_Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 style="text-align:center;font-size:200%;;">Pyspark для исследовательского анализа данных </h2>
<h3  style="text-align:center;">

<span class="label label-success">- Ленивая оценка</span>

<span class="label label-success">- Регистрация UDF</span>

<span class="label label-success">- Spark RDD</span>

<span class="label label-success">- Обработка данных</span>

<span class="label label-success">- Анализ данных</span>

<span class="label label-success">- Визуализация данных</span></h3>

## Преимущества изучения Pyspark

* Если компания планирует реализовать проект с действительно большим набором данных, Spark — <font color="red"><b>отличный выбор</b></font> .
Сравнительный анализ использования большого объема данных между `Pyspark` и `Pandas` по данным Датабрикс:  
![Spark_vs_Pandas.jpg](https://drive.google.com/uc?export=view&id=142bUyKI5BqOAkXRsYpOQoRCJ3YDOHS5R)


* Если вы планируете устроиться на работу в компанию с действительно большой экосистемой данных, знание Spark будет хорошим плюсом и отличает вас от других соискателей.


Ознакомиться с экосистемой `Spark`
 :https://www.kaggle.com/tientd95/pyspark-for-data-science

или сомтреть файл `pyspark-for-data-science.ipynb`

# **1. Инициализация платформы Spark и загрузка данных в фрейм данных Spark** <a class="anchor" id="1"></a>

[Go back to table of contents](#0.1)


Источник данных `1`: https://drive.google.com/file/d/1yiAp1fFDy3wSqUR0X_btCZPtuczbLwCe/view?usp=drive_link

Источник данных `2`: https://disk.yandex.ru/d/HKtMWSuqglgaXg

Этот набор данных предназначен для расчета частоты пульса у людей, а также других связанных с этим характеристик: пола, погодных условий, вида спорта, GPS и т.д.

In [ ]:
# Импорт модулей, не связанныех с PySpark.
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# Это помогает автоматически распечатывать элементы без явного использования «печати».
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Мага/Spark/3.2_PySpark/Data/endomondoHR.json.zip"

Archive:  /content/drive/MyDrive/Мага/Spark/3.2_PySpark/Data/endomondoHR.json.zip
  inflating: endomondoHR.json        


In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747919 sha256=5baa6fc06718bea2806be4f108aa6f8b5a4aec19141020590563e5a033ec25be
  Stored in directory: /root/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark


In [ ]:
# Импорт модулей, связанных с PySpark.
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

MAX_MEMORY = '15G'
# Инициализировать сеанс Spark.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Pyspark guide") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()
filename_data = 'endomondoHR.json' # Загрузите данные в текущий каталог Colab.
# filename_data = 'data/endomondoHR.json' # Загрузите данные в каталог data/  при работе в Docker.
# Загрузите основной набор данных в фрейм данных pyspark.
df = spark.read.json(filename_data, mode="DROPMALFORMED")
print('Data frame type: ' + str(type(df)))

Data frame type: <class 'pyspark.sql.dataframe.DataFrame'>


# **2. Обзор набора данных** <a class="anchor" id="2"></a>

[Go back to table of contents](#0.1)

### Схема, столбцы и типы данных набора:
   *Набор данных содержит как столбцы с одним значением (int, string), так и столбцы, состоящие из списка массивов.*

In [ ]:
print('Обзор данных')
df.printSchema()
print('Обзор столбцов')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Обзор данных
root
 |-- altitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- gender: string (nullable = true)
 |-- heart_rate: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- id: long (nullable = true)
 |-- latitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- longitude: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- speed: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- sport: string (nullable = true)
 |-- timestamp: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- url: string (nullable = true)
 |-- userId: long (nullable = true)

Обзор столбцов


,Column Name,Data type
0,altitude,array<double>
1,gender,string
2,heart_rate,array<bigint>
3,id,bigint
4,latitude,array<double>
5,longitude,array<double>
6,speed,array<double>
7,sport,string
8,timestamp,array<bigint>
9,url,string


In [ ]:
print('Описание фрейма данных (только строковые и числовые столбцы):')
df.describe().toPandas()

print(f'Общее количество {df.count()} строк, печатаем несколько первых строк:')
df.limit(2).toPandas()

Описание фрейма данных (только строковые и числовые столбцы):


,summary,gender,id,sport,url,userId
0,count,253020,253020,253020,253020,253020
1,mean,None,3.566244412926132E8,None,None,4619648.939783417
2,stddev,None,1.574845634895318E8,None,None,3932877.7296880498
3,min,female,99296,aerobics,https://www.endomondo.com/users/10014612/worko...,69
4,max,unknown,674008008,yoga,https://www.endomondo.com/users/9991401/workou...,15481421


Общее количество 253020 строк, печатаем несколько первых строк:


,altitude,gender,heart_rate,id,latitude,longitude,speed,sport,timestamp,url,userId
0,"[41.6, 40.6, 40.6, 38.4, 37.0, 34.0, 34.0, 34....",male,"[100, 111, 120, 119, 120, 116, 125, 128, 131, ...",396826535,"[60.173348765820265, 60.173239801079035, 60.17...","[24.64977040886879, 24.65014273300767, 24.6509...","[6.8652, 16.4736, 19.1988, 20.4804, 31.3956, 3...",bike,"[1408898746, 1408898754, 1408898765, 140889877...",https://www.endomondo.com/users/10921915/worko...,10921915
1,"[38.4, 39.0, 39.0, 38.2, 36.8, 36.8, 36.8, 35....",male,"[100, 105, 111, 110, 108, 115, 126, 130, 132, ...",392337038,"[60.173247596248984, 60.17320962622762, 60.172...","[24.649855233728886, 24.65015547350049, 24.650...","[9.0792, 13.284, 15.9336, 10.9476, 16.1676, 30...",bike,"[1408221682, 1408221687, 1408221699, 140822170...",https://www.endomondo.com/users/10921915/worko...,10921915


# **3. Обнаружение пропущенных значений и аномальных нулей.** <a class="anchor" id="3"></a>

[Go back to table of contents](#0.1)

После предварительного просмотра столбцов первое, что должны проверить, — это наличие в наборе данных какого-либо пропущенного значения.
– Для строковых столбцов  проверяем наличие `None` и `null`.
- Для числовых столбцов проверяем наличие нулей и `NaN`.
- Для столбцов типа массив проверяем, содержит ли массив нули или `NaN`.


In [ ]:
print('Обзор столбцов')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Обзор столбцов


,Column Name,Data type
0,altitude,array<double>
1,gender,string
2,heart_rate,array<bigint>
3,id,bigint
4,latitude,array<double>
5,longitude,array<double>
6,speed,array<double>
7,sport,string
8,timestamp,array<bigint>
9,url,string


In [ ]:
string_columns = ['gender', 'sport', 'url']
numeric_columns = ['id','userId']
array_columns = ['altitude', 'heart_rate', 'latitude', 'longitude', 'speed', 'timestamp']
missing_values = {}
for index, column in enumerate(df.columns):
    if column in string_columns:    # проверить столбцы строк со значениями None и Null
        missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values.update({column:missing_count})
    if column in numeric_columns:  # check zeroes, None, NaN
        missing_count = df.where(col(column).isin([0,None,np.nan])).count()
        missing_values.update({column:missing_count})
    if column in array_columns:  # check zeros and NaN
        missing_count = df.filter(array_contains(df[column], 0) | array_contains(df[column], np.nan)).count()
        missing_values.update({column:missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

,altitude,gender,heart_rate,id,latitude,longitude,speed,sport,timestamp,url,userId
0,40848,0,1280,0,113,113,7741,0,0,0,0


In [ ]:
#  создаем новый столбец для подсчета количества временных меток, записанных для каждой строки/тренировки, с именем столбца «PerWorkoutRecordCount».
df = df.withColumn('PerWorkoutRecordCount', size(col('timestamp')))


# Эта часть написана как функция, которую можно будет использовать позже.
def user_activity_workout_summarize(df):
    user_count = format(df.select('userId').distinct().count(), ',d')
    workout_count = format(df.select('id').distinct().count(), ',d')
    activity_count = str(df.select('sport').distinct().count())
    sum_temp = df.agg(functions.sum('PerWorkoutRecordCount')).toPandas()
    total_records_count = format(sum_temp['sum(PerWorkoutRecordCount)'][0],',d')
    columns=['Users count', 'Activity types count','Workouts count', 'Total records count']
    data = [[user_count], [activity_count], [workout_count], [total_records_count]]
    sum_dict = {column: data[i] for i, column in enumerate(columns)}
    sum_df = pd.DataFrame.from_dict(sum_dict)[columns]
    gender_user_count = df.select('gender','userId').distinct().groupBy('gender').count().toPandas()
    gender_activities_count = df.groupBy('gender').count().toPandas()
    gender_user_activity_count = gender_user_count.join(
        gender_activities_count.set_index('gender'), on='gender'
        , how='inner', lsuffix='_gu'
    )
    gender_user_activity_count.columns = ['Gender', '# of users', 'Activities (workouts) count']

    return sum_df, gender_user_activity_count

sum_dfs= user_activity_workout_summarize(df)
print('\nОбщая сводка набора данных о пользователях, тренировках и количестве записей (предварительная фильтрация):')
sum_dfs[0]


Общая сводка набора данных о пользователях, тренировках и количестве записей (предварительная фильтрация):


,Users count,Activity types count,Workouts count,Total records count
0,"1,104",49,"253,020","111,541,956"


Как мы видим, общее количество записей для этого набора данных превышает 111 миллионов записей. Это действительно большой размер

In [ ]:
print('Количество тренировок с менее чем 50 записями и статистической сводкой:')
removed_df = df.select('PerWorkoutRecordCount').where(df.PerWorkoutRecordCount < 50) \
               .toPandas().describe().astype(int)
removed_df.rename(columns = {'PerWorkoutRecordCount': 'PerWorkoutRecordCount <50'}, inplace=True)
removed_df.T

Количество тренировок с менее чем 50 записями и статистической сводкой:


,count,mean,std,min,25%,50%,75%,max
PerWorkoutRecordCount <50,5541,23,14,1,11,22,36,49


# **4. Ленивая оценка Pyspark** <a class="anchor" id="4"></a>

[Go back to table of contents](#0.1)

Здесь мы начнем знакомиться с некоторыми функциями продвинутого `Spark`: **Ленивая оценка**.

<font color="red"><b>Ленивая оценка</b></font> расширяет возможности Apache Spark за счет сокращения времени выполнения операций RDD. Он поддерживает граф происхождения для запоминания операций над RDD. мы можем просто помнить, что вся обработка в Pyspark — это абстракция. Когда мы хотим вернуть результаты, мы на самом деле сообщаем Spark, какой конечный ответ вас интересует, и он определяет лучший способ получить его. В результате это оптимизирует производительность и обеспечивает отказоустойчивость.

Чтобы увидеть результат, нам нужно вызвать `Spark.collect()`.

Обычно можем показать результаты с помощью синтаксиса: `df.take(k)` или `df.limit(k)`, чтобы получить результаты с `k`-строкой.

Когда `K` становится большим числом, выполнение этих двух способов, описанных выше, занимает много времени. Потому что приведенный выше синтаксис не использовал возможности обработки `Pyspark` (ленивая оценка).
Чтобы быстро обработать, требуется использовать `df.collect()[:k]` для возврата строки `k`, которая нам требуется.

Подробнее о ленивых вычислениях `Pyspark` можно прочитать в статье : https://data-flair.training/blogs/apache-spark-lazy-evaluation/

In [ ]:
ranked_sport_users_df = df.select(df.sport, df.userId) \
    .distinct() \
    .groupBy(df.sport) \
    .count() \
    .orderBy("count", ascending=False)

# Топ-5 типов тренировок
highest_sport_users_df = ranked_sport_users_df.limit(5).toPandas()
# Переименование имени столбца: 'count' -> Users count Количество пользователей.
highest_sport_users_df.rename(columns = {'count':'Users count'}, inplace = True)
# Подсчет общего количества пользователей. Результат потребуется для подсчета процентного соотношения.
total_sports_users = ranked_sport_users_df.groupBy().sum().collect()[0][0]

In [ ]:
ranked_sport_users_df.collect()[:5]

[Row(sport='run', count=865),
 Row(sport='bike', count=794),
 Row(sport='mountain bike', count=336),
 Row(sport='bike (transport)', count=252),
 Row(sport='walk', count=209)]

# **5. Исследовательский анализ данных** <a class="anchor" id="5"></a>

[Go back to table of contents](#0.1)

Давайте сначала составим таблицу, указав 5 лучших типов тренировок, которые мы оценили выше.

In [ ]:
# Calculate the percentage for the top 5 sports
highest_sport_users_df_renamed = highest_sport_users_df.copy()
highest_sport_users_df_renamed['percentage'] = highest_sport_users_df['Users count'] / total_sports_users * 100

# Creating the 'others' group
others = {
    'sport': 'others',
    'Users count': total_sports_users - sum(highest_sport_users_df_renamed['Users count']),
    'percentage': 100 - sum(highest_sport_users_df_renamed['percentage'])
}

# Convert 'others' into a DataFrame
others_df = pd.DataFrame([others])

# Use pd.concat() to append 'others' to the original DataFrame
highest_sport_users_df_renamed = pd.concat([highest_sport_users_df_renamed, others_df], ignore_index=True)

print('Топ-5 видов спорта, в которых участвует больше всего пользователей:')
print(highest_sport_users_df_renamed)

# Plotting
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=plt.figaspect(0.35))

# Bar plot
axs[0].bar(x=highest_sport_users_df_renamed['sport'], height=highest_sport_users_df_renamed['Users count'])
axs[0].set_title('Вид спортивных увлечений', fontsize='small')
axs[0].set_xlabel('Вид спорта', fontsize='small')
axs[0].set_ylabel('Количество пользователей', fontsize='small')
axs[0].set_xticklabels(highest_sport_users_df_renamed['sport'], rotation='vertical', fontsize='small')

# Pie chart
explode = (0.1, 0.1, 0.3, 0.3, 0.3, 0.1)
axs[1].pie(
    x=highest_sport_users_df_renamed['percentage'],
    labels=highest_sport_users_df_renamed['sport'],
    autopct='%1.1f%%', shadow=True, explode=explode, startangle=90, radius=1
)
axs[1].set_title('Вид спортивных увлечений', fontsize='small')

# Adding text to the figure
fig.text(0.5, 1.02, 'Топ-5 видов спорта, в которых участвует больше всего пользователей', ha='center', va='top', transform=fig.transFigure)

# Show the plot
plt.show()

NameError: name 'highest_sport_users_df_renamed' is not defined

Данные показывают, что пользователи тратят наибольшую часть затрат на занятия, связанные с бегом, ходьбой и ездой на велосипеде, что вполне разумно, поскольку эти упражнения удобны без особых вложений 👍

In [ ]:
# Просмотр данных в зависимости от пола
activities_by_gender = df.groupBy('sport', 'gender').count().toPandas()
activities_by_gender[:5]

# **6. UNSTACK PYSPARK DATAFRAME** <a class="anchor" id="5"></a>
[Go back to table of contents](#0.1)

Требуется изменить форму приведенной выше таблицы, чтобы разбить столбец пол, для последующей визуализации.

Определим принцип работы с категориями:

- метод `DataFrame.stack()`: "сводит" уровень (возможно, иерархический) меток столбцов, возвращая a `DataFrame` с новым, самым внутренним уровнем меток строк;

- метод `DataFrame.unstack()`: (обратная операция `DataFrame.stack()`) "сводит" уровень индекса строки (возможно, иерархический) к оси столбца, создавая измененный `DataFrame` с новым, самым внутренним уровнем меток столбцов.

![Spark_vs_Pandas.jpg](https://drive.google.com/uc?export=view&id=1OLmDoS68kKpPLj1bFAPoXKZEjJJcrAeM)

Чтобы изменить форму таблицы в Pyspark, мы используем
```python
df.unstack()
```
Относительно к нашим данным получим следующее разбиение:

![Spark_vs_Pandas.jpg](https://drive.google.com/uc?export=view&id=1lS8DOC-DfsKQOcI0YekPR1lapKpPhXah)



In [ ]:
total_activities = ranked_sport_users_df.count()
print(f'Всего: {total_activities} активности в зависимости от пола:')
# Добавление информации о занятиях в зависимости от пола
activities_by_gender = df.groupBy('sport', 'gender').count().toPandas()
# Визуализация
fig = plt.figure(figsize=(12, 25))
grid_size = (1,1);
ax = plt.subplot2grid(grid_size, (0,0), colspan=1, rowspan=1)
plot = activities_by_gender.groupby(['sport', 'gender']).agg(np.mean).groupby(level=0).apply(
    lambda x: 100 * x / x.sum()).unstack().plot(kind='barh', stacked=True, width=1  ## APPLY UNSTACK TO RESHAPE DATA
                , edgecolor='black', ax=ax, title='Список всех занятий в зависимости от пола')
ylabel = plt.ylabel('Sport (Activity)');
xlabel = plt.xlabel('Процент участия по гендерному признаку');
legend = plt.legend(
    sorted(activities_by_gender['gender'].unique()), loc='center left', bbox_to_anchor=(1.0, 0.5)
)
param_update = plt.rcParams.update({'font.size': 16});
ax = plt.gca()
formatter = ax.xaxis.set_major_formatter(mtick.PercentFormatter());
a = fig.tight_layout()
plt.show()

Давайте посмотрим на верхнее парето из 5 видов спорта, в которых больше всего участников.

In [ ]:
# Pivot the data by gender and sport
activities_by_gender_df = activities_by_gender.pivot_table(
    index="sport", columns="gender", values='count', fill_value=0) \
    .reset_index().rename_axis(None, axis=1)

# Add total count and percentage
activities_by_gender_df['total'] = activities_by_gender_df['male'] + activities_by_gender_df['female'] + activities_by_gender_df['unknown']
activities_by_gender_df['percentage'] = activities_by_gender_df['total'] / sum(activities_by_gender_df['total']) * 100

# Get top 5 activities by percentage
top_activities_by_gender_df = activities_by_gender_df.sort_values(by='percentage', ascending=False).head(5)

# Create the 'others' group
others = {'sport': 'others'}
for column in ['female', 'male', 'unknown', 'total', 'percentage']:
    value = sum(activities_by_gender_df[column]) - sum(top_activities_by_gender_df[column])
    others.update({column: value})

# Convert 'others' to a DataFrame
others_df = pd.DataFrame([others])

# Use pd.concat to add 'others' row to the DataFrame
top_activities_by_gender_df = pd.concat([top_activities_by_gender_df, others_df], ignore_index=True)

# Sort by percentage again
top_activities_by_gender_df = top_activities_by_gender_df.sort_values(by='percentage', ascending=False)

# Display the final DataFrame
print(top_activities_by_gender_df)

# Plotting
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=plt.figaspect(0.35))

# Bar plot for total count of activities
axs[0].bar(x=top_activities_by_gender_df['sport'], height=top_activities_by_gender_df['total'])
axs[0].set_title('Количество тренировок', fontsize='small')
axs[0].set_xlabel('Спорт', fontsize='small')
axs[0].set_ylabel('Количество тренировок (раз)', fontsize='small')
axs[0].set_xticklabels(top_activities_by_gender_df['sport'], rotation='vertical', fontsize='small')

# Pie chart for percentage distribution
explode = (0.1, 0.1, 0.3, 0.3, 0.3, 0.3)
axs[1].pie(
    x=top_activities_by_gender_df['percentage'],
    labels=top_activities_by_gender_df['sport'],
    autopct='%1.1f%%', shadow=True, explode=explode, radius=1
)
axs[1].set_title('Соотношение тренировок', fontsize='small')

# Add text to the figure
fig.text(0.5, 1.02, 'Топ-5 видов спорта', ha='center', va='top', transform=fig.transFigure)

# Show the plot
plt.show()

Опять же, как и в случае с участием пользователей, бег, ходьба и поездки на велосипеде также являются основными факторами, для определния  количества тренировок. Однако единственное отличие состоит в том, что количество занятий бегом и ездой на велосипеде намного больше, чем количество остальных видов спорта, и общее количество этих двух видов уже составляет более 85% от общего количества занятий.

Давайте поиграем с каким-нибудь вопросом, например, сколько людей участвовало более чем в одном виде спорта.


In [ ]:
min_number_of_sports = 1

sport_df = df \
    .select(df.userId, df.gender, df.sport) \
    .distinct() \
    .groupBy(df.userId, df.gender) \
    .count()

user_more_sports_df = sport_df \
                    .filter(sport_df["count"] > min_number_of_sports) \
                    .orderBy("count", ascending = False) \
                    .toPandas()
user_more_sports_df.rename(columns = {'count':'Sports count'}, inplace = True)
user_more_sports_df.describe().astype(int).T

*По итогам 822 человека занимались более чем 1 видом спорта. Среди них в среднем человек занимается примерно 3 видами спорта, а есть кто занимается до 16 видами спорта!* <br />
Теперь посмотрим на статистику по полу на коробчатой диаграмме:


In [ ]:
plot = user_more_sports_df.boxplot(column='Sports count', by='gender', fontsize='small', figsize=(6,7))

*график показал, что, за исключением выбросов, и мужчины, и женщины имеют почти одинаковое распределение участия в спорте..*

### Распределение количества рекордов за тренировку

*Для более детального наблюдения мы разбили количество рекордов по каждому виду деятельности по каждому отдельному виду спорта. <br />Исходя из распределения, максимальное количество записей за тренировку составляет 500, но не все тренировки и виды спорта достигают этого числа.*

In [ ]:
print('\nГрафик распределения тренировок по видам деятельности:')
plot_size_x, plot_size_y = 5, 5
figsize_x, figsize_y = plot_size_x * 4 + 3, plot_size_y * 13 + 1
figsize=(figsize_x, figsize_y)
fig = plt.figure(figsize=figsize) #
grid_size = (13,4)
ax = plt.subplot2grid(grid_size, (0,0), colspan=1, rowspan=1)
#fig, ax = plt.subplots()
PerWorkoutRecordCount_dist = df.select('PerWorkoutRecordCount', 'sport').toPandas().hist(
    column='PerWorkoutRecordCount', bins=10, sharex = False, grid=True
    , xlabelsize='small', ylabelsize='small', by='sport', ax = ax
    , layout = grid_size, figsize=figsize
    )
a = fig.tight_layout()
title = fig.text(0.5, 1, 'Распределение количества рекордов за тренировку по видам спорта', ha='center'
         , fontsize='small', transform=fig.transFigure);
xlabel = fig.text(
    0.5, 0.01, '# рекорды/тренировки', va='bottom', ha='center', transform=fig.transFigure
)
ylabel = fig.text(0.01, 0.5, 'Частота (количество)', va='center', rotation='vertical');

In [ ]:
# Отфильтровать df с минимум 10 записями (поскольку мы предполагаем, что любой user_id с записью менее 10 не будет иметь смысла)
qualified_df = df \
    .select(df.sport, df.userId, df.gender) \
    .groupBy(df.sport, df.userId, df.gender) \
    .count()
qualified_df = qualified_df.filter(qualified_df["count"] >= 10) \
    .orderBy("count", ascending = False)

In [ ]:
print('Количество пользователей, имеющих более 10 тренировок:')
qualified_pd_df = qualified_df.select("userId", "gender").distinct() \
    .groupBy(qualified_df.gender).count().toPandas()
qualified_pd_df.rename(columns={'count': 'Users count'}, inplace=True)
qualified_pd_df
qualified_users_count = sum(qualified_pd_df['Users count'])
total_users_count = df.select('userId').distinct().count()
qualified_percentage = round((qualified_users_count / total_users_count),2) * 100
print('\nТаким образом, есть {} / {} пользователей, соответствующих 10 критериям исторических записей, что {:.2f}%' \
      .format(qualified_users_count, total_users_count, qualified_percentage)
     )

# **7. Pyspark UDF** <a class="anchor" id="6"></a>
[Go back to table of contents](#0.1)

Что такое `UDF`?

`User Defined Functions` - это пользовательские функции. Если  знакомы с `SQL`, `UDF` не являются для вас чем-то новым, поскольку большинство традиционных баз данных РСУБД поддерживают пользовательские функции, эти функции необходимо зарегистрировать в библиотеке базы данных и использовать их в `SQL` как обычные функции.

`UDF PySpark` аналогичны `UDF` в традиционных базах данных. В `PySpark` создается функция в синтаксисе `Python` и обертываете ее `PySpark SQL udf()` или регистрируете ее как `udf` и используете ее в `DataFrame` и `SQL` соответственно.

Зачем нужна `UDF`?

`UDF` используются для расширения функций платформы и повторного использования этих функций в нескольких `DataFrame`. Например, требуется преобразовать каждую первую букву слова в строке имени в заглавную; Встроенные функции `PySpark` не имеют этой функции, поэтому можно создать `UDF` и повторно использовать ее по мере необходимости во многих кадрах данных. После создания `UDF` их можно повторно использовать в нескольких выражениях `DataFrame` и `SQL`.

Прежде чем создавать какую-либо пользовательскую функцию, изучите, доступна ли аналогичная функция, которая вам нужна, в `Spark SQL Functions` . `PySpark SQL` предоставляет несколько предопределенных общих функций, и с каждым выпуском добавляется множество новых функций, следовательно, лучше всего проверить, прежде чем изобретать велосипед.

При создании `UDF`  необходимо проектировать их очень тщательно, иначе столкнетесь с проблемами оптимизации и производительности.

Короче говоря, регистрация UDF в Pyspark — это процесс **превращения функций Python в функции PySpark (UDF)**.

![Spark_vs_Pandas.jpg](https://drive.google.com/uc?export=view&id=1x5s0p9oa-dNIsNo5XzYQjdMQUxt99WYP)

Когда запускаем код в кластерах `Spark`, этот метод ускорит процесс и сэкономит время выполнения.

Чтобы узнать больше о `Pyspark UDF`, посетите https://changhsinlee.com/pyspark-udf/.

Выполним регистрацию функции Python к UDF в столбце `timestamp`.


Этот столбец очень важен, если используем этот набор данных для прогнозирования, например, для прогнозирования частоты пульса, чтобы определить возможные проблемы с сердцем у пользователя фитнес-трекера.

### Создание некоторых новых функций из `timestamp`

Столбец `timestamp` содержит записи серий временных меток одной тренировки (строка данных) и хранится в формате временных меток `UNIX`. Чтобы получить больше информации об этом столбце,  создадим из него `4` новых столбца:
- `date_time`: конвертировать временную метку `UNIX` в формат даты и времени Python.
- `duration`: общее время одной тренировки в минутах.
   *Чтобы получить «продолжительность» тренировки, получаем разницу между максимальным и минимальным значением списка даты и времени каждой тренировки.*
- `workout_start_time`: определим, когда и в какой час дня начинается тренировка.
   *Для «workout_start_time» это первая часть записи даты и времени тренировки.*
- `interval`: список промежутков времени между каждой отдельной записью временной метки в одной тренировке, в секундах.
   *Что касается интервала, рассчитаем его, взяв разницу между двумя последовательными записями временных меток в течение тренировки.*

Чтобы прикрепить эти 4 функции к `SparkdDataFrame`,  регистрируем их с помощью `PYSPARK UDF`.

```python
function_to_udf = udf(function, Datatype())

# Datatype() can be floattype(), TimestampType(), etc

```


In [3]:
# ПОСМОТРИТЕ СНОВА КОЛОНКУ ВРЕМЕННОЙ МЕТКИ
df.limit(3).toPandas()

NameError: name 'df' is not defined

### ### Мы создаем 4 вспомогательные функции для столбца 'timestamp', как описано выше, а затем преобразуем их в UDF.

In [2]:
# Преобразование столбца метки времени в Datetime.Datetime, чтобы позже использовать его для функции .withColumn.
def to_time(timestamp_list):
    # преобразовать в дату и время и минус 7 часов из-за разницы во временном окне Endomondo с временем utc в качестве описания набора данных
    return [datetime.fromtimestamp(t) - timedelta(hours=7) for t in timestamp_list]

# Регистрация вспомогательной функции to_time в структуре UDF pyspark
udf_to_time = udf(to_time, ArrayType(elementType=TimestampType()))

# Вспомогательная функция для получения продолжительности (в минутах) списка значений даты и времени, которая будет позже использована для функции withColumn.
def get_duration(datetime_list):
    time_dif = max(datetime_list) - min(datetime_list)
    return time_dif.seconds/60

# Регистрация вспомогательной функции get_duration как пользовательской функции в pyspark.
udf_get_duration = udf(get_duration, FloatType())

# Вспомогательная функция для получения времени начала тренировки из списка даты и времени, которая будет позже использована для функции withColumn.
def get_start_time(datetime_list):
    return min(datetime_list)

# Регистрация вспомогательной функции get_start_time как пользовательской функции в pyspark
udf_get_start_time = udf(get_start_time, TimestampType())

# Вспомогательная функция для получения списка интервалов во время тренировки
def get_interval(datetime_list):
    if len(datetime_list) == 1:
        return [0]
    else:
        interval_list = []
        for i in range(0, len(datetime_list)-1):
            interval = (datetime_list[i+1] - datetime_list[i]).seconds
            interval_list.append(interval)
        return interval_list

# Регистрация вспомогательной функции get_interval как пользовательской функции в pyspark.
udf_get_interval = udf(get_interval, ArrayType(elementType=IntegerType()))

# Создание нового столбца date_time для преобразования метки времени в формат даты и времени Python для последующего использования.
df = df.withColumn('date_time',
    udf_to_time('timestamp'))

# Создание столбца «workout_start_time», чтобы получить время начала каждой тренировки/строки:
df = df.withColumn('workout_start_time', hour(udf_get_start_time('date_time')))

# Создание столбца продолжительности из только что созданного столбца date_time, используя функцию udf udf_get_duration, определенную выше.
df = df.withColumn('duration', udf_get_duration('date_time'))

# Создание столбца интервала из столбца date_time, используя функцию udf udf_get_interval, определенную выше.
df = df.withColumn('interval', udf_get_interval('date_time'))

print('Новые столбцы (''date_time'', ''workout_start_time'' in hour\
, ''duration'' in minutes & ''interval'' in seconds)\n, first 5 rows:')
df.select('timestamp','date_time', 'workout_start_time', 'duration', 'interval').limit(5).toPandas()



NameError: name 'udf' is not defined

### Теперь посмотрим на продолжительность каждой тренировки (в минутах).

In [ ]:
print('\nСтатистика длительности (в минутах):')
df.select('duration').toPandas().describe().T

Из статистики столбца `duration` видно, что продолжительность тренировки может длиться от 0 минут до 1 полного дня (1440 минут = 24 часа). Продолжительность 0 может быть для тренировок, которые имеют только одну запись, поэтому минимальная и максимальная отметки времени будут одинаковыми.<br />
Теперь пришло время построения графика продолжительности:


In [ ]:

print('\nПостроение графика распределения продолжительности по видам спорта:')
plot_size_x, plot_size_y = 5, 5
figsize_x, figsize_y = plot_size_x * 4 +3, plot_size_y * 13 + 1
figsize = (figsize_x, figsize_y)
fig = plt.figure(figsize=figsize) #
grid_size = (13,4)
ax = plt.subplot2grid(grid_size, (0,0), colspan=1, rowspan=1)

duration_dist = df.select('duration', 'sport').toPandas().hist(
    column='duration', by='sport', bins=15, sharex = False, grid=True
    , xlabelsize='small', ylabelsize='small' , ax = ax
    , layout = grid_size, figsize=figsize
    )
a = fig.tight_layout()
title = fig.text(0.5, 1, 'Распределение продолжительности тренировок по видам спорта'
             , ha='center', va='center', transform=fig.transFigure
            )
xlabel = fig.text(0.5, 0.01, 'Продолжительность тренировки (минуты)'
             , ha='center', va='center', transform=fig.transFigure)
ylabel = fig.text(0.01, 0.5, 'Частота (количество)', va='center', rotation='vertical');

* Судя по сводке продолжительности и графику распределения, большинство занятий происходит в течение 1–2 часов, только несколько видов спорта и несколько случаев каждого типа происходят в течение более длительных периодов, таких как поездка на горном велосипеде, пеший туризм, парусный спорт и т. д. *

# **8.Преобразование объектов строк в устойчивый распределенный набор данных Spark (RDD)** <a class="anchor" id="7"></a>
[Go back to table of contents](#0.1)

На этом графике мы попрактикуемся в преобразовании объекта строки в формат RDD в Pyspark с помощью:

```python
rdd = df.rdd.map(tuple)
or
rdd = df.rdd.map(list)
```
Расширенный формат RDD: каждый набор данных разделен на логические части, которые можно легко вычислить на разных узлах кластера. Они могут работать параллельно и являются отказоустойчивыми, поэтому процесс является стабильным и очень быстрым.

Если мы запустим этот код, например, на Zeppelin, который интегрирован с кластерами Pyspark, мы увидим, какова скорость `RDD Spark`.

`RDD` — очень важная концепция в `Spark`, можете углубиться в нее здесь:
https://www.educba.com/what-is-rdd/

Чтобы попрактиковаться в этой концепции,  посмотрим на столбец `interval` и получим по нему некоторую статистику. Рассчитаем некоторые основные статистические данные (мин/макс/среднее/среднее/стандартное отклонение и 4 квантиля 25/50/75/95) в `pySpark`, преобразуем в Rdd и построим их на графике.



In [ ]:
# Вспомогательная функция для расчета статистики имени столбца из кортежа x (спорт, список записей столбца) tuple x of (sport, records list of the column)
#, статистика для расчета также предоставляется в качестве входных данных
def calculate_stats(x,column_name, stat_list):
    sport, records_list = x
    stat_dict = {'sport': sport}
    if 'min' in stat_list:
        min_stat = min(records_list)
        stat_dict.update({'min ' + column_name : min_stat})
    if 'max' in stat_list:
        max_stat = max(records_list)
        stat_dict.update({'max ' + column_name: max_stat})
    if 'mean' in stat_list:
        average_stat = stats.mean(records_list)
        stat_dict.update({'mean ' + column_name: average_stat})
    if 'stdev' in stat_list:
        std_stat = stats.stdev(records_list)
        stat_dict.update({'stdev ' + column_name: std_stat})
    if '50th percentile' in stat_list:
        median_stat = stats.median(records_list)
        stat_dict.update({'50th percentile ' + column_name: median_stat})
    if '25th percentile' in stat_list:
        percentile_25th_stat = np.percentile(records_list, 25)
        stat_dict.update({'25th percentile ' + column_name: percentile_25th_stat})
    if '75th percentile' in stat_list:
        percentile_75th_stat = np.percentile(records_list, 75)
        stat_dict.update({'75th percentile ' + column_name: percentile_75th_stat})
    if '95th percentile' in stat_list:
        percentile_95th_stat = np.percentile(records_list, 95)
        stat_dict.update({'95th percentile ' + column_name: percentile_95th_stat})
    return stat_dict

def to_list(a):
    return a

def extend(a, b):
    a.extend(b)
    return a

def retrieve_array_column_stat_df(df, column_name, stat_list):
    # Преобразование спорт и «column_name» в RDD, чтобы легко рассчитать статистику интервалов по видам спорта.
    sport_record_rdd = df.select('sport', column_name).rdd \
    .map(tuple).combineByKey(to_list, extend, extend).persist()

    # Вычислить статистику входного столбца, вызвав функцию Calcul_stats, определенную выше.
    record_statistic_df = pd.DataFrame(sport_record_rdd.map(
        lambda x: calculate_stats(x, column_name,stat_list)).collect()
                                      )
    # Установка правильного порядка столбцов данных.
    columns_order = ['sport'] + [stat + ' ' + column_name for stat in stat_list]
    # Изменение порядка столбцов
    return record_statistic_df[columns_order]

stat_list = ['min', '25th percentile', 'mean', '50th percentile',
                     '75th percentile', '95th percentile', 'max', 'stdev']
interval_statistic_df = retrieve_array_column_stat_df(df, column_name='interval', stat_list=stat_list)
print('\nПросмотр статистики интервалов в секундах (по виду спорта)' )
interval_statistic_df

Теперь мы отображаем эти числа в виде столбцов (для статистики квантилей) и линейных диаграмм (для мин/макс/среднего/стандартного отклонения) для большей наглядности.
*Примечание. Поскольку максимальный интервал и стандартное отклонение имеют гораздо более высокий порядок величины по сравнению с остальными столбцами, нам нужно поместить эти два столбца на отдельную ось Y справа.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print('\nОбобщенная статистика интервальных видов спорта:')

bar_columns = ['25th percentile interval', '50th percentile interval', '75th percentile interval', '95th percentile interval']
line_columns1 = ['min interval', 'mean interval']
line_columns2 = ['max interval', 'stdev interval']

interval_statistic_df = interval_statistic_df.sort_values(by='95th percentile interval', ascending=False)

figsize = (13, 59)
fig, axs = plt.subplots(nrows=7, figsize=figsize)

d = axs[0].set_title('Интервальная статистика по видам спорта', fontsize=18)

for i in range(7):
    interval_statistic_sub_df = interval_statistic_df.iloc[i*7:i*7+7]

    # Plot the bar chart for the quantiles
    plot1 = interval_statistic_sub_df[['sport'] + bar_columns].groupby(['sport']).agg(np.mean).plot(
        kind='bar', stacked=True, grid=False, alpha=0.5, edgecolor='black', ax=axs[i]
    )

    # Plot the line chart for min and mean intervals
    plot2 = interval_statistic_sub_df[['sport'] + line_columns1].plot(x='sport', ax=axs[i], marker='o')

    # Create a secondary y-axis for max and stdev intervals
    ax2 = axs[i].twinx()
    plot3 = interval_statistic_sub_df[['sport'] + line_columns2].plot(x='sport', ax=ax2, marker='o', color=['m', 'g'])

    # Legends for the plots
    a = axs[i].legend(loc='center left', fontsize=16, bbox_to_anchor=(1.2, 0.5), frameon=False)
    a = ax2.legend(labels=['max interval (right)', 'stdev interval (right)'], loc="center left", fontsize=16, bbox_to_anchor=(1.2, 0.11), frameon=False)

    # Formatting for x-ticks and labels
    b = axs[i].set_xticklabels(interval_statistic_sub_df['sport'], rotation='horizontal', fontsize='small')
    c = axs[i].set_xlabel('Спорт (Активность)', fontsize='small')
    d = axs[i].set_ylabel('Статистика квантилей + мин/среднее\n(секунд)', fontsize=16)
    e = ax2.set_ylabel('Max/stdev Statistics\n(second)', fontsize=16)

    # Set font size for y-axis ticks
    for tick in axs[i].yaxis.get_major_ticks():
        tick.label1.set_fontsize(16)  # Use label1 for primary y-axis labels
    ax2.tick_params(axis='y', labelsize=16)

    # Make sure x-tick labels are visible for all subplots
    b = plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

# Adjust the layout for better visualization
plt.subplots_adjust(hspace=0.2)

# Show the plot
plt.show()

до 95% набора интервальных данных не имеют интервала более 400 секунд, в то время как есть лишь несколько выбросов, из-за которых максимальные интервалы достигают 86400 секунд (полных дней).

### Время начала тренировки
Мы снова используем гистограмму, чтобы посмотреть на распределение часов начала тренировок, сгруппированных по видам спорта и с разбивкой по полу. Мы делим день на интервалы по 2 часа, всего получается 12 частей.

In [ ]:
# Получите таблицу пола, вида спорта и времени начала тренировки для построения графика.
start_time_df = df.select('gender', 'sport','workout_start_time').toPandas()

In [ ]:
activities = start_time_df['sport'].unique()
plot_size_x, plot_size_y = 5, 5
figsize_x, figsize_y = (plot_size_x + 0.5) * 4 +3, (plot_size_y + 1) * 13 + 1


nrows, ncols = 13, 4
a = fig.subplots_adjust(hspace = 1, wspace = 1)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(figsize_x, figsize_y))
print('\nГрафик распределения времени начала тренировки по видам спорта с разбивкой по полу:')
a = plt.setp(axs, xticks=[0, 4, 8, 12, 16, 20])
for index, sport in enumerate(activities):
    row_index, col_index = divmod(index, ncols)
    male_start_time_list = start_time_df[(start_time_df.sport == sport) &
                                            (start_time_df.gender == 'male')]['workout_start_time']
    female_start_time_list = start_time_df[(start_time_df.sport == sport) &
                                            (start_time_df.gender == 'female')]['workout_start_time']
    unknown_start_time_list = start_time_df[(start_time_df.sport == sport) &
                                            (start_time_df.gender == 'unknown')]['workout_start_time']
    if len(male_start_time_list) > 0:
        male_dist = axs[row_index, col_index].hist(male_start_time_list,
                                      bins = 12, alpha=0.5, label='male', range=(0, 23))
    if len(female_start_time_list) > 0:
        female_dist = axs[row_index, col_index].hist(female_start_time_list,
                                      bins = 12, alpha=0.5, label='female', range=(0, 23))
    if len(unknown_start_time_list) > 0:
        unknown_dist = axs[row_index, col_index].hist(unknown_start_time_list,
                                      bins = 12, alpha=0.5, label = 'unknown', range=(0, 23))
    b= axs[row_index, col_index].set_title('Activitiy: ' + sport, fontsize='small')
    a = axs[row_index, col_index].legend(loc="upper left", fontsize='small')
    a = plt.setp(axs[row_index, col_index].get_xticklabels(), fontsize='small')

for i in range(1,4):
    x = axs[12, i].set_visible(False)
a = fig.tight_layout()
z = fig.text(0.5, 1, 'Распределение времени начала тренировки (часы) по видам спорта'
             , ha='center', va='top', transform=fig.transFigure)
y = fig.text(0.5, 0.01, 'Начало тренировки час в день (час)'
             , ha='center', va='bottom', transform=fig.transFigure)
z = fig.text(0.02, 0.5, 'Частота (количество)', va='center', rotation='vertical');

Из приведенных выше диаграмм распределения видно, что в большинстве видов спорта занятия начинаются либо утром, либо вечером (бимодальное распределение), что имеет смысл. В период с 0 до 4 часов происходит несколько событий, что довольно странно.
<br />

### Посмотрите глубже на информацию на уровне строки

In [ ]:
stat_list = ['min', '25th percentile', 'mean', '95th percentile', 'max', 'stdev']
heart_rate_statistic_df = retrieve_array_column_stat_df(df, column_name='heart_rate', stat_list=stat_list)

*Из-за огромного количества пользователей и количества тренировок мы случайным образом выбрали до x количества пользователей каждого пола (например, 5) и до y тренировок по каждому типу активности (например, 10).<br />*

In [ ]:
# Вспомогательная функция, которая помогает отбирать данные
def sampling_data(max_users_per_gender, max_workouts_per_sport):
    '''
          max_users_per_gender: максимальное количество пользователей, выбираемых случайным образом для каждого пола.
          max_workouts_per_sport: максимальное количество занятий, которые можно выбрать для каждого вида спорта.
          (виды спорта, существующие у выбранных пользователей)
    '''
    # Получение уникального списка идентификаторов пользователя и пола для выборки.
    users_genders = df.select('userId', 'gender').distinct().toPandas()
    #Выбор до трех идентификаторов пользователей каждого пола из уникального списка идентификаторов пользователей.
    random_x_users_per_gender = users_genders.groupby('gender')['userId'].apply(
                lambda s: s.sample(min(len(s), max_users_per_gender))
    )

    # Применение фильтра к pyspark dataframe для выборки.
    samples_by_gender = df.where(df.userId.isin(list(random_x_users_per_gender)))

    # Генерация уникальных идентификаторов занятий и списков видов спорта из выборочного набора данных.
    workout_sports = samples_by_gender.select('id', 'sport').distinct().toPandas()
    # выбор до 10 идентификаторов занятий для каждого вида спорта.
    random_y_workouts_per_sport = workout_sports.groupby('sport')['id'].apply(
        lambda s: s.sample(min(len(s), max_workouts_per_sport))
    )

    # фильтр к выборочному набору данных, чтобы продолжить сокращать количество тренировок для каждого типа активности.
    samples_by_gender_and_sport = samples_by_gender.where(df.id.isin(list(random_y_workouts_per_sport)))
    return samples_by_gender_and_sport

In [ ]:
# Использование 2 переменных для определения критериев выборки:
# максимальное количество пользователей по полу и максимальное количество тренировок по виду спорта
max_users_per_gender, max_workouts_per_sport = 20, 15

# Сбор набора выборочных данных в Pandas для использования с функциями графика.
pd_df = sampling_data(max_users_per_gender, max_workouts_per_sport).toPandas()
print('\nОбзор выборочных данных (только строковые и числовые столбцы)):')
pd_df.describe()

нормализуем время для всех тренировок, рассчитав продолжительность (в секундах) каждой записи временной метки из первой записи тренировки (первый элемент datetime списка в этой тренировке). <br />
Затем отображаем частоту сердечных сокращений в зависимости от этого нормализованного времени, группируя по видам спорта.

In [ ]:
# Лямбда-функция для объединения списка списков в один большой список
flattern = lambda l: set([item for sublist in l for item in sublist])

normalized_datetime_list = []
for index,data_row in pd_df.iterrows():
    min_date_time = min(data_row['date_time'])
    normalized_datetime_list.append(
        [(date_time - min_date_time).seconds for date_time in data_row['date_time']]
    )

pd_df['normalized_date_time'] = normalized_datetime_list

print('New normalized datetime (first 7 rows):')
pd_df.head(7)[['userId', 'sport', 'date_time','normalized_date_time']]

print('\nПостроение необработанного пульс (выборка) по нормированному времени:')

sport_list = pd_df['sport'].unique()
# Динамическое определение длины фигуры зависит от длины спортивного списка.
fig, axs = plt.subplots(len(sport_list), figsize=(15, 6*len(sport_list)))
subplot_adj = fig.subplots_adjust(hspace = 0.6)
plot_setp = plt.setp(axs, yticks=range(0,250,20))

for sport_index, sport in enumerate(sport_list):
    workout = pd_df[pd_df.sport == sport]
    max_time = max(flattern(workout.normalized_date_time))
    for workout_index, data_row in workout.iterrows():
        label = 'user: ' + str(data_row['userId']) + ' - gender: ' + data_row['gender']
        plot_i = axs[sport_index].plot(
            data_row['normalized_date_time'], data_row['heart_rate'], label=label
        )
    title_i = axs[sport_index].set_title('Activitiy: ' + sport, fontsize='small')
    xlabel_i = axs[sport_index].set_xlabel('Time (sec)', fontsize='small')
    xsticklabels_i = axs[sport_index].set_xticklabels(
        range(0, max_time, 500),rotation = 'vertical', fontsize=9
    )
    ysticklabels_i = axs[sport_index].set_yticklabels(range(0,250,20),fontsize='small')
    legend_i = axs[sport_index].legend(
        loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'size': 9}
    )

x_label = fig.text(0.04, 0.5, 'Частота сердечных сокращений (уд/мин)', va='center', rotation='vertical')
chart_title = fig.text(0.5, 1.3, 'Необработанная частота пульса (выборка) по нормализованному времени',
            ha='center', va='center', fontsize='small', transform=axs[0].transAxes)


### Перемещения во время тренировки
У нас будет некоторая визуализация в трех столбцах с информацией о смещении/геометрии (`longitude`,`latitude` и `altitude`).
Поскольку  расположение каждого пользователя и тренировки отличается друг от друга, мы отображаем только несколько отдельных тренировок на 3D-графиках, чтобы просмотреть маршрут тренировки.


In [ ]:
pd_df_small = sampling_data(max_users_per_gender=2, max_workouts_per_sport=2).toPandas()
print('Sampled data (2 user, 2 workouts per sport):')
pd_df_small[['userId', 'gender','sport','id', 'workout_start_time'
             ,'PerWorkoutRecordCount', 'duration', 'longitude', 'latitude', 'altitude']].describe()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.ticker as mtick
import numpy as np  # To use numpy functions for standard deviation

def get_fixed_mins_maxs(mins, maxs):
    deltas = (maxs - mins) / 12.
    mins = mins + deltas / 4.
    maxs = maxs - deltas / 4.
    return [mins, maxs]

workout_count = pd_df_small.shape[0]
ncols = 3
nrows = math.ceil(workout_count / ncols)

fig = plt.figure(figsize=(8 * (ncols + 0.5), 8 * nrows))
fig.subplots_adjust(hspace=0.2, wspace=0.5)

print('Построение траекторий тренировки в виде 3D для каждой тренировки:')
for row_index, row in pd_df_small.iterrows():
    min_long = min(row['longitude']) - np.std(row['longitude'])  # Use numpy.std()
    max_long = max(row['longitude']) + np.std(row['longitude'])  # Use numpy.std()
    minmax_long = get_fixed_mins_maxs(min_long, max_long)

    min_lat = min(row['latitude']) - np.std(row['latitude'])  # Use numpy.std()
    max_lat = max(row['latitude']) + np.std(row['latitude'])  # Use numpy.std()
    minmax_lat = get_fixed_mins_maxs(min_lat, max_lat)

    min_alt = min(row['altitude']) - np.std(row['altitude'])  # Use numpy.std()
    max_alt = max(row['altitude']) + np.std(row['altitude'])  # Use numpy.std()
    minmax_alt = get_fixed_mins_maxs(min_alt, max_alt)

    ax = fig.add_subplot(nrows, ncols, row_index + 1, projection='3d')

    title = 'Активность: ' + row['sport'] + ' - Пол: ' + row['gender'] + \
            '\nРекорды: ' + str(int(row['PerWorkoutRecordCount'])) + \
            ' - Длительность: ' + str(int(row['duration'])) + ' минуты'
    ax.set_title(title, fontsize=16)

    # Scatter plot for points
    scatter = ax.scatter(row['longitude'], row['latitude'], row['altitude'], c='r', marker='o')

    # Plot the workout path in 3D
    plot = ax.plot3D(row['longitude'], row['latitude'], row['altitude'], c='gray', label='Workout path')

    # Labels for the axes
    ax.set_xlabel('Долгота (градусы)', fontsize=16)
    ax.set_ylabel('Широта (градусы)', fontsize=16)
    ax.set_zlabel('Высота (м)', fontsize=16, rotation=0)

    # Set font size for ticks (accessing `label1` for the primary ticks)
    for t in ax.xaxis.get_major_ticks():
        t.label1.set_fontsize(16)
    for t in ax.yaxis.get_major_ticks():
        t.label1.set_fontsize(16)
    for t in ax.zaxis.get_major_ticks():
        t.label1.set_fontsize(16)

    # Add legends
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

    # Set limits for each axis
    ax.set_xlim(minmax_long)
    ax.set_ylim(minmax_lat)
    if minmax_alt[0] != minmax_alt[1]:
        ax.set_zlim(minmax_alt)

    # Set aspect ratio for the plot (adjust these values as needed)
    ax.set_box_aspect([4, 2, 0.5])  # aspect ratio for x, y, z axes

    # Remove spines
    for spine in ax.spines.values():
        spine.set_visible(False)

# Add a global title
fig.text(0.5, 1.02, "Маршрут тренировки (долгота/широта/высота)", ha='center', va='top', fontsize=18)

plt.tight_layout()
plt.show()